In [ ]:
import os
import glob


data_root = "/data/shared/cityscapes/leftImg8bit/train/"
assert os.path.exists(data_root)
data_subdirs = next(os.walk(data_root))[1]

mmdet_root = "../submodules/lib-avstack-core/third_party/mmdetection"
mmseg_root = "../submodules/lib-avstack-core/third_party/mmsegmentation"

## Show an example from the carla dataset

In [ ]:
def get_sample_image_path():
    sample_img_path = glob.glob(os.path.join(data_root, data_subdirs[0], "*.png"))[5]
    return sample_img_path

In [ ]:
from PIL import Image, ImageOps
import cv2
import numpy as np

sample_img_path = get_sample_image_path()

# load image
img_in = Image.open(sample_img_path)
img_in = ImageOps.exif_transpose(img_in)  # do not transpose automatically
bbox = None
img_array = np.array(img_in)[:, :, ::-1]

# add bounding box to image (x, y, w, h)
if bbox is not None:
    img = cv2.rectangle(
        img_array,
        (bbox[0], bbox[1]),
        (bbox[0] + bbox[2], bbox[1] + bbox[3]),
        color=(255, 0, 0),
        thickness=20,
    )
    img = Image.fromarray(img)
else:
    img = Image.fromarray(img_array)

# scale to a manageable size
rescale = True
if rescale:
    base_width = 600
    wpercent = base_width / float(img.size[0])
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((base_width, hsize), Image.BICUBIC)

# display the image with jupyter's builtin display
display(img)

## Test MMSegmentation

In [ ]:
from mmseg.apis import MMSegInferencer

sample_img_path = get_sample_image_path()

# Choose to use a config
cfg = "deeplabv3plus_r50-d8_4xb2-40k_cityscapes-512x1024"
config = os.path.join(mmseg_root, f"configs/deeplabv3plus/{cfg}.py")
checkpoint = os.path.join(
    mmseg_root,
    "checkpoints/Cityscapes/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth",
)

# Set the device to be used for evaluation
device = "cuda:0"

# Initialize the MMSegInferencer
inferencer = MMSegInferencer(model=config, weights=checkpoint, device=device)

# Use the detector to do inference
result = inferencer(sample_img_path, out_dir="./output_mmseg", show=True)

## Test MMDetection

In [ ]:
from mmdet.apis import DetInferencer

sample_img_path = get_sample_image_path()

# Choose to use a config
cfg = "faster-rcnn_r50_fpn_1x_cityscapes"
config = os.path.join(mmdet_root, f"configs/cityscapes/{cfg}.py")
checkpoint = os.path.join(
    mmdet_root,
    "checkpoints/cityscapes/faster_rcnn_r50_fpn_1x_cityscapes_20200502-829424c0.pth",
)

# Set the device to be used for evaluation
device = "cuda:0"

# Initialize the DetInferencer
inferencer = DetInferencer(config, checkpoint, device)

# Use the detector to do inference
result = inferencer(sample_img_path, out_dir="./output_mmdet")